In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
from PIL import Image
import gradio as gr


[notice] A new release of pip is available: 24.3.1 -> 25.0
[notice] To update, run: C:\Users\dubey\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/57.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/57.7 MB ? eta -:--:--
   ---------------------------------------- 0.3/57.7 MB ? eta -:--:--
   ---------------------------------------- 0.5/57.7 MB 730.2 kB/s eta 0:01:19
   ---------------------------------------- 0.5/57.7 MB 730.2 kB/s eta 0:01:19
    --------------------------------------- 0.8/57.7 MB 730.2 kB/s eta 0:01:18
    --------------------------------------- 1.0/57.7 MB 812.4 kB/s eta 0:01:10
    --------------------------------------- 1.0/57.7 MB 812.4 kB/s eta 0:01:10
    --------------------------------------- 1.0/57.7 MB 812.4 kB/s eta 0:01:10
    --------------------------------------- 1.3/57.7 MB 737.9 kB/s eta 0:01:17
   - -------------------------------------- 1.6/57.7 MB 777.0 kB/s eta 0:01:13
   - -------------------------------------- 1.8/57.7 MB 792.8 kB/s eta 0:01:11
 

In [5]:
def load_utkface_data(dataset_path, img_size=(128, 128)):
    images, ages, genders = [], [], []
    
    for filename in os.listdir(dataset_path):
        if filename.endswith(".jpg"):
            parts = filename.split("_")
            age = int(parts[0])
            gender = int(parts[1])  # 0 = male, 1 = female
            
            # Load and preprocess image
            img = Image.open(os.path.join(dataset_path, filename)).convert("RGB")
            img = img.resize(img_size)
            img = np.array(img) / 255.0  # Normalize to [0, 1]
            
            # Append to lists
            images.append(img)
            ages.append(age)
            genders.append(gender)
    
    # Convert to numpy arrays
    images = np.array(images)
    ages = np.array(ages)
    genders = np.array(genders)
    
    return images, ages, genders


In [6]:
def create_hair_labels(ages, genders):
    hair_length = []
    for age, gender in zip(ages, genders):
        if 20 <= age <= 30:
            hair_length.append(gender)  # Long hair for females, short for males
        else:
            hair_length.append(1 - gender)  # Opposite for outside age range
    return np.array(hair_length)

In [8]:
dataset_path = "UTKFace" 
images, ages, genders = load_utkface_data(dataset_path)
hair_length = create_hair_labels(ages, genders)

In [9]:
X_train, X_test, y_age_train, y_age_test, y_gender_train, y_gender_test, y_hair_train, y_hair_test = train_test_split(
    images, ages, genders, hair_length, test_size=0.2, random_state=42
)
X_train, X_val, y_age_train, y_age_val, y_gender_train, y_gender_val, y_hair_train, y_hair_val = train_test_split(
    X_train, y_age_train, y_gender_train, y_hair_train, test_size=0.2, random_state=42
)


In [10]:
def build_model(input_shape=(128, 128, 3)):
    inputs = tf.keras.Input(shape=input_shape)
    
    # Shared CNN backbone
    x = layers.Conv2D(32, (3, 3), activation='relu')(inputs)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Conv2D(64, (3, 3), activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = layers.Flatten()(x)
    x = layers.Dense(128, activation='relu')(x)
    
    # Task-specific heads
    age = layers.Dense(1, activation='linear', name='age')(x)
    gender = layers.Dense(1, activation='sigmoid', name='gender')(x)
    hair_length = layers.Dense(1, activation='sigmoid', name='hair_length')(x)
    
    return models.Model(inputs, [age, gender, hair_length])

model = build_model()
model.compile(
    optimizer='adam',
    loss={'age': 'mse', 'gender': 'binary_crossentropy', 'hair_length': 'binary_crossentropy'},
    metrics={'age': 'mae', 'gender': 'accuracy', 'hair_length': 'accuracy'}
)

In [11]:
history = model.fit(
    X_train,
    {'age': y_age_train, 'gender': y_gender_train, 'hair_length': y_hair_train},
    validation_data=(X_val, {'age': y_age_val, 'gender': y_gender_val, 'hair_length': y_hair_val}),
    epochs=20,
    batch_size=32
)


Epoch 1/20
475/475 ━━━━━━━━━━━━━━━━━━━━ 80s 155ms/step - age_loss: 359.1006 - age_mae: 14.3367 - gender_accuracy: 0.6240 - gender_loss: 0.6826 - hair_length_accuracy: 0.5533 - hair_length_loss: 0.7328 - loss: 360.5151 - val_age_loss: 172.1141 - val_age_mae: 9.6542 - val_gender_accuracy: 0.7723 - val_gender_loss: 0.4746 - val_hair_length_accuracy: 0.5150 - val_hair_length_loss: 0.7178 - val_loss: 173.5326
Epoch 2/20
475/475 ━━━━━━━━━━━━━━━━━━━━ 68s 144ms/step - age_loss: 165.1929 - age_mae: 9.6715 - gender_accuracy: 0.7734 - gender_loss: 0.5016 - hair_length_accuracy: 0.5731 - hair_length_loss: 0.6889 - loss: 166.3841 - val_age_loss: 133.4062 - val_age_mae: 8.5005 - val_gender_accuracy: 0.8350 - val_gender_loss: 0.3757 - val_hair_length_accuracy: 0.5530 - val_hair_length_loss: 0.6890 - val_loss: 134.3704
Epoch 3/20
475/475 ━━━━━━━━━━━━━━━━━━━━ 69s 144ms/step - age_loss: 121.2821 - age_mae: 8.1903 - gender_accuracy: 0.8088 - gender_loss: 0.4207 - hair_length_accuracy: 0.6103 - hair_lengt

In [12]:
def predict_gender(image):
    # Preprocess image
    img = np.array(image.resize((128, 128))) / 255.0
    img = np.expand_dims(img, axis=0)
    
    age_pred, gender_pred, hair_pred = model.predict(img)
    
    if 20 <= age_pred <= 30:
        gender = "Female" if hair_pred > 0.5 else "Male"
    else:
        gender = "Female" if gender_pred > 0.5 else "Male"
    
    return f"Predicted Gender: {gender}, Age: {int(age_pred)}"


In [17]:
model.save("gender_age_hair_model.h5")